# TrackMan Data Engineering Challenge: Table Dependencies Graph

In [1]:
import os
import json

def get_queries(directory):
    queries = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                query = json.load(file)
                queries += query
    return queries

In [3]:
def get_table_dependencies(query):

    table_dependency = {}

    schema = query['schema']['S']
    table = query['table']['S']
    new_table = schema + '.' + table

    from_tables = []
    from_query = query['query']['L'][0]['M']['from']['S']
    from_query = from_query.split()
    from_tables.append(from_query[0])
    for string in from_query:
        if string.lower() == 'join':
            table = from_query[from_query.index(string)+1]
            if table not in from_tables:
                from_tables.append(table)
            del from_query[from_query.index(string)]

    table_dependency[new_table] = from_tables

    return table_dependency


In [4]:
def print_table_dependencies(table_dependencies)

SyntaxError: expected ':' (3568406111.py, line 1)

In [ ]:
def get_individual_dependency(query)